# Imports

In [1]:
import firebase_admin
from firebase_admin import firestore
from firebase_admin import credentials

import logging
import traceback
from pprint import pformat
from typing import Dict, Optional, Union, List, Tuple

from geopy.distance import geodesic

import numpy as np
from collections import Counter
from sklearn.cluster import DBSCAN

## Parameters

In [2]:
creds_path: str = "./keys/serviceAccountKey.json"
max_distance_km: float = 5.0

## Setup

In [3]:
logging.basicConfig(
    level=logging.INFO, format="%(levelname)s: %(asctime)s - %(message)s"
)

logger = logging.getLogger(__name__)

## Authentication

In [4]:
try:
    cred = credentials.Certificate(creds_path)
    logger.info("Firebase centantial read successfully")
    firebase_admin.initialize_app(cred)
    logger.info("Firebase initialized successfully")
    db = firestore.client()
    logger.info("Firestore client created")

except Exception as e:
    traceback_str = traceback.format_exc()
    logger.error("An error occurred: %s", str(e))
    logger.debug(f"Traceback: {traceback_str}")

INFO: 2024-01-21 14:44:42,541 - Firebase centantial read successfully
INFO: 2024-01-21 14:44:42,542 - Firebase initialized successfully
INFO: 2024-01-21 14:44:42,542 - Firestore client created


## Custom Error

In [5]:
class MarkerNotFoundError(Exception):
    pass

## Utils

In [ ]:
def getAllMarkers(firebase_clent):
    markers = []

    try:
        markers_get = firebase_clent.collection("Markers").get()
        logger.info("Successfully retrived all Markers")

    except Exception as e:
        traceback_str = traceback.format_exc()
        logger.error("An error occurred: %s", str(e))
        logger.debug(f"Traceback: {traceback_str}")

    for m in markers_get:  # type: ignore
        marker = {
            "marker-id": str(m.id),
            "marker_cord": (m.to_dict()["lat"], m.to_dict()["long"]),
        }
        markers.append(marker)

    logger.info("Successfully got all Markers into python dict")
    return markers

In [ ]:
def find_markers_within_distance(
    markers: list, target_coord: tuple, max_distance_km: float = 5.0
):
    nearby_markers = []

    try:
        for marker in markers:
            marker_coord = marker["marker_cord"]
            distance = geodesic(target_coord, marker_coord).kilometers

            if distance <= max_distance_km:
                nearby_markers.append(marker)
        logger.debug(
            "Successfully looped over all markers and selected markers within distance"
        )
    except Exception as e:
        traceback_str = traceback.format_exc()
        logger.error("An error occurred: %s", str(e))
        logger.debug(f"Traceback: {traceback_str}")

    logger.info(f"Successfully got nearby markers: {len(nearby_markers)}")
    return nearby_markers

In [ ]:
def cluster_markers_fn(markers, epsilon=1.0, min_samples=2):
    """
    Cluster markers by coordinates using DBSCAN.

    Parameters:
    - markers: List of dictionaries, each containing 'marker-id' and 'marker_cord'.
    - epsilon: Maximum distance between two samples for one to be considered in the neighborhood of the other.
    - min_samples: The number of samples in a neighborhood for a point to be considered a core point.

    Returns:
    - A modified list of markers with an additional 'cluster_label' field.
    """
    # Extract coordinates from markers
    coordinates = np.array([marker["marker_cord"] for marker in markers])

    # Perform DBSCAN clustering
    dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)

    # Add cluster labels to the markers
    for i, marker in enumerate(markers):
        marker["cluster_label"] = labels[i]

    return markers

In [19]:
# def get_cluster_label(marker_data, target_marker_id):
#     for marker in marker_data:
#         if marker["marker-id"] == target_marker_id:
#             return marker["cluster_label"]
#     raise MarkerNotFoundError(f"Marker ID {target_marker_id} not found in the list.")

In [ ]:
# def upadteClusterIDfirestore(firebaseClient, clusters: list[dict]):
#     marker_ids = [marker["marker-id"] for marker in clusters]
#     try:
#         for key in marker_ids:
#             ref = firebaseClient.collection("Markers").document(key)
#             ref.update(
#                 {
#                     "cluster": int(get_cluster_label(clusters, key)),
#                 }
#             )
#         logger.info("Added updated cluster ids")
#     except Exception as e:
#         traceback_str = traceback.format_exc()
#         logger.error("An error occurred: %s", str(e))
#         logger.debug(f"Traceback: {traceback_str}")

In [ ]:
def find_nearest_marker(markers, target_lat, target_lon, max_distance_km=5.0):
    target_location = (target_lat, target_lon)
    nearest_marker = None
    min_distance = float("inf")

    for marker in markers:
        marker_location = marker["marker_cord"]
        distance = geodesic(target_location, marker_location).kilometers

        if distance < min_distance and distance <= max_distance_km:
            min_distance = distance
            nearest_marker = marker

    return nearest_marker

## Program

In [ ]:
markers_data = getAllMarkers(db)
logger.info(f"Total markers retrived {len(markers_data)}")
logger.debug(pformat(markers_data))

In [ ]:
clustered_markers = cluster_markers_fn(markers_data, epsilon=0.1, min_samples=1)

In [ ]:
clustered_markers

In [ ]:
upadteClusterIDfirestore(firebaseClient=db, clusters=clustered_markers)

## testing

In [ ]:
import folium
from folium.plugins import MarkerCluster

# Create a folium map centered around India
india_map = folium.Map(location=[20.5937, 78.9629], zoom_start=5)

# Create a MarkerCluster layer
marker_cluster = MarkerCluster().add_to(india_map)

# Add markers to the map
for marker in clusters:  # type: ignore
    folium.Marker(
        location=marker["marker_cord"],
        popup=f"Marker ID: {marker['marker-id']}, Cluster: {marker['cluster_label']}",
    ).add_to(marker_cluster)

# Save the map to an HTML file or display it
india_map

## class

In [24]:
class statsNearYou:
    def __init__(
        self,
        firebase_clent,
        max_distance_km: float,
    ) -> None:
        """
        Initialize the StatsNearYou instance.

        Parameters:
        - firebase_client : Firebase client instance.
        - max_distance_km (float): Maximum distance in kilometers to consider for finding the nearest marker.
        """
        self.firebaseClient = firebase_clent
        self.getAllMarkers()
        self.getAllSOS()
        self.max_distance_km = max_distance_km
        self.cluster_markers_fn()
        self.cluster_SOS_Reports_fn()
        logger.info("Creadted statsNearYou object.")
        self.rate_clusters()
        self.rate_SOS_clusters()

    def get_SOS_cluster_label(self, target_marker_id: str) -> int:
        """
        Get the cluster label for a given marker ID.

        Parameters:
        - target_marker_id (str): Marker ID.

        Returns:
        - int: Cluster label.
        """
        for marker in self.SOS_Reports:
            if marker["SOS_Reports-id"] == target_marker_id:
                logger.info("Successful got cluster label for given marker.")
                return marker["cluster_label"]
        raise MarkerNotFoundError(
            f"Marker ID {target_marker_id} not found in the list."
        )

    def get_cluster_label(self, target_marker_id: str) -> int:
        """
        Get the cluster label for a given marker ID.

        Parameters:
        - target_marker_id (str): Marker ID.

        Returns:
        - int: Cluster label.
        """
        for marker in self.markers:
            if marker["marker-id"] == target_marker_id:
                logger.info("Successful got cluster label for given marker.")
                return marker["cluster_label"]
        raise MarkerNotFoundError(
            f"Marker ID {target_marker_id} not found in the list."
        )

    def getAllSOS(
        self,
    ) -> None:
        """
        Retrieve all SOSReports from Firebase and store them in the instance.
        """
        SOS_Reports = []
        try:
            SOS_Reports_get = self.firebaseClient.collection("SOS_Reports").get()
            logger.info("Successfully retrived all SOS_Reports")

        except Exception as e:
            traceback_str = traceback.format_exc()
            logger.error("An error occurred: %s", str(e))
            logger.debug(f"Traceback: {traceback_str}")

        for m in SOS_Reports_get:  # type: ignore
            SOS_Report = {
                "SOS_Reports-id": m.id,
                "SOS_Reports_cord": (
                    m.to_dict()["incidentPosition"].to_protobuf().latitude,
                    m.to_dict()["incidentPosition"].to_protobuf().longitude,
                ),
            }
            SOS_Reports.append(SOS_Report)
        logger.info("Successfully got all SOS_Reports into python dict")
        self.SOS_Reports = SOS_Reports

    def getAllMarkers(
        self,
    ) -> None:
        """
        Retrieve all markers from Firebase and store them in the instance.
        """
        markers = []
        try:
            markers_get = self.firebaseClient.collection("Markers").get()
            logger.info("Successfully retrived all Markers")

        except Exception as e:
            traceback_str = traceback.format_exc()
            logger.error("An error occurred: %s", str(e))
            logger.debug(f"Traceback: {traceback_str}")

        for m in markers_get:  # type: ignore
            marker = {
                "marker-id": m.to_dict()["id"],
                "marker_cord": (m.to_dict()["lat"], m.to_dict()["long"]),
            }
            markers.append(marker)
        logger.info("Successfully got all Markers into python dict")
        self.markers = markers

    def cluster_SOS_Reports_fn(self) -> None:
        """
        Cluster SOS_Reports by coordinates using DBSCAN.
        """
        coordinates = np.array(
            [marker["SOS_Reports_cord"] for marker in self.SOS_Reports]
        )
        # Perform DBSCAN clustering
        labels = dbscan.fit_predict(coordinates)
        logger.info("Clustered SOS_Reports successfully.")

        # Add cluster labels to the markers
        for i, marker in enumerate(self.SOS_Reports):
            marker["cluster_label"] = labels[i]
        logger.info("Updated Clusters with cluster ids.")

    def cluster_markers_fn(self) -> None:
        """
        Cluster markers by coordinates using DBSCAN.
        """
        coordinates = np.array([marker["marker_cord"] for marker in self.markers])
        # Perform DBSCAN clustering
        labels = dbscan.fit_predict(coordinates)
        logger.info("Clustered makeres successfully.")

        # Add cluster labels to the markers
        for i, marker in enumerate(self.markers):
            marker["cluster_label"] = labels[i]
        logger.info("Updated Clusters with cluster ids.")

    def getClusteredSOS_Reports(self) -> list[dict]:
        return self.SOS_Reports

    def getClusteredMarkers(self) -> list[dict]:
        return self.markers

    def upadteClusterIDSOSfirestore(self) -> None:
        """
        Update cluster IDs in Firestore for all markers.
        """
        marker_ids = [marker["SOS_Reports-id"] for marker in self.SOS_Reports]
        try:
            for key in marker_ids:
                ref = self.firebaseClient.collection("SOS_Reports").document(key)
                ref.update(
                    {
                        "cluster": int(self.get_SOS_cluster_label(key)),
                    }
                )
            logger.info("Added updated cluster ids")
        except Exception as e:
            traceback_str = traceback.format_exc()
            logger.error("An error occurred: %s", str(e))
            logger.debug(f"Traceback: {traceback_str}")

    def upadteClusterIDfirestore(self) -> None:
        """
        Update cluster IDs in Firestore for all markers.
        """
        marker_ids = [marker["marker-id"] for marker in self.markers]
        try:
            for key in marker_ids:
                ref = self.firebaseClient.collection("Markers").document(key)
                ref.update(
                    {
                        "cluster": int(self.get_cluster_label(key)),
                    }
                )
            logger.info("Added updated cluster ids")
        except Exception as e:
            traceback_str = traceback.format_exc()
            logger.error("An error occurred: %s", str(e))
            logger.debug(f"Traceback: {traceback_str}")

    def find_nearest_SOS_Reports(
        self, target_lat: float, target_lon: float
    ) -> Optional[Dict[str, Union[str, Tuple[float, float], int]]]:
        """
        Find the nearest SOS_Reports within the specified maximum distance.

        Parameters:
        - target_lat (float): Target latitude.
        - target_lon (float): Target longitude.

        Returns:
        - Optional[Dict[str, Union[str, Tuple[float, float], int]]]: Nearest marker information or None if not found.
        """
        target_location = (target_lat, target_lon)
        nearest_marker = None
        min_distance = float("inf")

        for marker in self.SOS_Reports:
            marker_location = marker["SOS_Reports_cord"]
            distance = geodesic(target_location, marker_location).kilometers

            if distance < min_distance and distance <= self.max_distance_km:
                min_distance = distance
                nearest_marker = marker
        logger.info("Returned nearest SOS_Reports successfully.")
        return nearest_marker

    def find_nearest_marker(
        self, target_lat: float, target_lon: float
    ) -> Optional[Dict[str, Union[str, Tuple[float, float], int]]]:
        """
        Find the nearest marker within the specified maximum distance.

        Parameters:
        - target_lat (float): Target latitude.
        - target_lon (float): Target longitude.

        Returns:
        - Optional[Dict[str, Union[str, Tuple[float, float], int]]]: Nearest marker information or None if not found.
        """
        target_location = (target_lat, target_lon)
        nearest_marker = None
        min_distance = float("inf")

        for marker in self.markers:
            marker_location = marker["marker_cord"]
            distance = geodesic(target_location, marker_location).kilometers

            if distance < min_distance and distance <= self.max_distance_km:
                min_distance = distance
                nearest_marker = marker
        logger.info("Returned nearest markers successfully.")
        return nearest_marker

    def get_total_SOS_Reports_in_each_cluster(self):
        cluster_counts = Counter(marker["cluster_label"] for marker in self.SOS_Reports)
        return dict(cluster_counts)

    def get_total_markers_in_each_cluster(self):
        cluster_counts = Counter(marker["cluster_label"] for marker in self.markers)
        return dict(cluster_counts)

    def calculate_percentile(self, dictionary, key):
        values = list(dictionary.values())
        values_below = sum(value < dictionary[key] for value in values)
        total_values = len(values)

        percentile = (values_below / total_values) * 100

        return percentile

    def rate_SOS_clusters(self):
        cluster_counts = self.get_total_SOS_Reports_in_each_cluster()
        total_markers = len(self.SOS_Reports)
        # Calculate percentile for each cluster
        cluster_percentiles = {
            key: self.calculate_percentile(cluster_counts, key)
            for key in cluster_counts
        }
        logger.info("Successfully calullated SOS percentiles")
        # Assign star ratings based on percentiles
        cluster_ratings = {}
        for cluster_label, percentile in cluster_percentiles.items():
            if percentile >= 80:
                rating = 1
            elif 60 <= percentile < 80:
                rating = 2
            elif 40 <= percentile < 60:
                rating = 3
            elif 20 <= percentile < 40:
                rating = 4
            elif 0 <= percentile < 20:
                rating = 4
            cluster_ratings[cluster_label] = rating  # type:ignore

        self.SOS_cluster_ratings = cluster_ratings
        logger.info("Successfully Assigned SOS ratings")
        # print(type(cluster_ratings))

        try:
            doc_ref = self.firebaseClient.collection("Stats")
            for key, value in cluster_ratings.items():
                doc_ref.document(f"SOS_{key}").set({"SOS_Reports_star": value})
            logger.info("Successfully Updated to FireStoere")
        except Exception as e:
            traceback_str = traceback.format_exc()
            logger.error("An error occurred: %s", str(e))
            logger.debug(f"Traceback: {traceback_str}")

    def rate_clusters(self):
        cluster_counts = self.get_total_markers_in_each_cluster()
        total_markers = len(self.markers)
        # Calculate percentile for each cluster
        cluster_percentiles = {
            key: self.calculate_percentile(cluster_counts, key)
            for key in cluster_counts
        }
        logger.info("Successfully calullated percentiles")
        # Assign star ratings based on percentiles
        cluster_ratings = {}
        for cluster_label, percentile in cluster_percentiles.items():
            if percentile >= 80:
                rating = 1
            elif 60 <= percentile < 80:
                rating = 2
            elif 40 <= percentile < 60:
                rating = 3
            elif 20 <= percentile < 40:
                rating = 4
            elif 0 <= percentile < 20:
                rating = 4
            cluster_ratings[cluster_label] = rating  # type:ignore

        self.cluster_ratings = cluster_ratings
        logger.info("Successfully Assigned ratings")
        # print(type(cluster_ratings))

        try:
            doc_ref = self.firebaseClient.collection("Stats")
            for key, value in cluster_ratings.items():
                doc_ref.document(f"{key}").set({"marker_star": value})
            logger.info("Successfully Updated to FireStoere")
        except Exception as e:
            traceback_str = traceback.format_exc()
            logger.error("An error occurred: %s", str(e))
            logger.debug(f"Traceback: {traceback_str}")

    def statsByCoord(
        self, lat: float, lon: float
    ) -> Optional[Dict[str, Union[str, Tuple[float, float], int]]]:
        """
        Retrieve statistics for a given set of coordinates.

        Parameters:
        - lat (float): Latitude.
        - lon (float): Longitude.

        Returns:
        - Optional[Dict[str, Union[str, Tuple[float, float], int]]]: Statistics for the given coordinates or None if not found.
        """
        marker = self.find_nearest_marker(target_lat=lat, target_lon=lon)
        SOS = self.find_nearest_SOS_Reports(target_lat=lat, target_lon=lon)
        try:
            data = (
                self.firebaseClient.collection("Stats")
                .document(f"{marker['cluster_label']}")  # type: ignore
                .get()
            )
            data2 = (
                self.firebaseClient.collection("Stats")
                .document(f"SOS_{marker['cluster_label']}")  # type: ignore
                .get()
            )
            logger.info("successfully got Statf for given coord")
            res_dict = data.to_dict()
            res_dict.update(data2.to_dict())
            return res_dict
        except Exception as e:
            traceback_str = traceback.format_exc()
            logger.error("An error occurred: %s", str(e))
            logger.debug(f"Traceback: {traceback_str}")

In [25]:
stats = statsNearYou(firebase_clent=db, max_distance_km=5.0)

INFO: 2024-01-21 15:13:30,299 - Successfully retrived all Markers
INFO: 2024-01-21 15:13:30,307 - Successfully got all Markers into python dict
INFO: 2024-01-21 15:13:30,546 - Successfully retrived all SOS_Reports
INFO: 2024-01-21 15:13:30,547 - Successfully got all SOS_Reports into python dict
INFO: 2024-01-21 15:13:30,550 - Clustered makeres successfully.
INFO: 2024-01-21 15:13:30,551 - Updated Clusters with cluster ids.
INFO: 2024-01-21 15:13:30,555 - Clustered SOS_Reports successfully.
INFO: 2024-01-21 15:13:30,556 - Updated Clusters with cluster ids.
INFO: 2024-01-21 15:13:30,557 - Creadted statsNearYou object.
INFO: 2024-01-21 15:13:30,558 - Successfully calullated percentiles
INFO: 2024-01-21 15:13:30,559 - Successfully Assigned ratings
INFO: 2024-01-21 15:13:31,278 - Successfully Updated to FireStoere
INFO: 2024-01-21 15:13:31,279 - Successfully calullated SOS percentiles
INFO: 2024-01-21 15:13:31,280 - Successfully Assigned SOS ratings
INFO: 2024-01-21 15:13:31,405 - Successfu

In [26]:
stats.SOS_Reports

[{'SOS_Reports-id': 'E5QUaafGwfQ5E28VTV0y',
  'SOS_Reports_cord': (26.1405954, 91.7623148),
  'cluster_label': 0},
 {'SOS_Reports-id': 'ki7bh24dYXgNDELoDsRI',
  'SOS_Reports_cord': (26.1405001, 91.7623384),
  'cluster_label': 0},
 {'SOS_Reports-id': 'p02uTWo4DCEkK2USTduS',
  'SOS_Reports_cord': (26.138171213188684, 91.75585884600878),
  'cluster_label': 0}]

In [27]:
stats.statsByCoord(lat=26.120600, lon=91.652300)

INFO: 2024-01-21 15:13:33,026 - Returned nearest markers successfully.
INFO: 2024-01-21 15:13:33,031 - Returned nearest SOS_Reports successfully.
INFO: 2024-01-21 15:13:33,289 - successfully got Statf for given coord


{'marker_star': 2, 'SOS_Reports_star': 4}